In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import ks_2samp
from scipy.special import stdtr


def helpFloat(x): 
    try:
        return float ( float(x['thumbsup']) / (float(x['thumbsup']) + float(x['thumbsdown'])) )
    except:
        return 0
    
def thumbsSum(x): 
    try:
        return int(x['thumbsup']) + int(x['thumbsdown']) 
    except:
        return 0

def sentiment(x): 
    if x['stars'] == 5:
        return 'favorable'
    if x['stars'] == 1:
        return 'unfavorable'
    else:
        return 'mixed'
    
reviews_features = pd.read_csv('reviews.csv.gz', compression='gzip')
reviews_features['helpfulness'] = reviews_features.apply(helpFloat,axis=1)
reviews_features['thumbstotal'] = reviews_features.apply(thumbsSum,axis=1)
reviews_features = reviews_features[reviews_features.thumbstotal>4]
reviews_features = reviews_features[reviews_features.word_count>9]
reviews_features = reviews_features[np.isfinite(reviews_features.helpfulness)]
list_search_categories = [ 'Celular e Smartphone' , 'TV', 'Camera Digital', 'Maquina de Lavar Roupas'
                          , 'Geladeira / Refrigerador', 'Ar Condicionado', 'Tablet', 'Notebook'
                          ,'Console de Videogame', 'Impressora', 'Fogao', 'Microondas'
                          , 'Aparelho de Telefone', 'MP3 Player / MP4 Player', 'Aquecedor de Ambiente'
                          , 'Forno Eletrico']
reviews_features = reviews_features[reviews_features.category.isin(list_search_categories)]
reviews_features.shape

(7444, 49)

## split dataset sentiment

In [10]:
mixed_reviews = reviews_features[reviews_features.stars > 1]
mixed_reviews = mixed_reviews[reviews_features.stars < 5]
favorable_reviews = reviews_features[reviews_features.stars == 5]
unfavorable_reviews = reviews_features[reviews_features.stars == 1]

## t-student test

In [11]:
def tstudent(df1, df2):
    col =list(df1)
    count=1
    for i in xrange(len(col)-1): 
        try:
            # Create sample data.
            a = df1[col[i]].as_matrix()
            b = df2[col[i]].as_matrix()
            # Use scipy.stats.ttest_ind.
            t, p = ttest_ind(a, b, equal_var=False)
            if (p <= 0.05):
                #print p
                if (np.mean(a) > np.mean(b)):
                    dife="scientific > non_scientific"
                else:
                    dife="scientific < non_scientific"

                print "%i. %s t = %f  p = %.4g  %s" % (count, dife,  t, p, col[i])

                count=count+1
            else:
                print "IGUAL %s p=%f" %(col[i],p)
        except:
            print "ERROR %s " %(col[i])

## favorable x unfavorable

In [12]:
tstudent(favorable_reviews, unfavorable_reviews)

1. scientific > non_scientific t = 6.192726  p = 9.948e-10  Unnamed: 0
IGUAL ConnectiveAdditiveIncidence p=0.734745
IGUAL sentence_sized_30 p=0.223197
IGUAL redability p=0.814728
IGUAL mean_sentence_length p=0.171871
2. scientific < non_scientific t = -2.004392  p = 0.04542  percentile_75_sentence_length
3. scientific > non_scientific t = 2.470721  p = 0.01367  percentile_75_word_length
4. scientific < non_scientific t = -9.229879  p = 3.103e-19  word_count
IGUAL ConnectiveLogicIncidence p=0.482387
IGUAL percentile_90_word_length p=0.204028
5. scientific > non_scientific t = 2.675829  p = 0.007563  mean_word_length
6. scientific < non_scientific t = -9.218459  p = 3.62e-19  syllable_count
IGUAL avg_word_per_sentence p=0.213321
IGUAL LogicIfIncidence p=nan
IGUAL percentile_50_sentence_length p=0.571777
IGUAL median_sentence_length p=0.571777
IGUAL LogicAndIncidence p=0.482387
7. scientific < non_scientific t = -3.481081  p = 0.0005303  percentile_90_sentence_length
IGUAL median_word_len

## favorable x mixed

In [13]:
tstudent(favorable_reviews, mixed_reviews)

IGUAL Unnamed: 0 p=0.507477
IGUAL ConnectiveAdditiveIncidence p=0.400764
1. scientific < non_scientific t = -3.690984  p = 0.0002277  sentence_sized_30
IGUAL redability p=0.353477
2. scientific < non_scientific t = -2.892443  p = 0.003853  mean_sentence_length
3. scientific < non_scientific t = -3.930128  p = 8.705e-05  percentile_75_sentence_length
IGUAL percentile_75_word_length p=0.262711
4. scientific < non_scientific t = -14.218311  p = 1.282e-44  word_count
IGUAL ConnectiveLogicIncidence p=0.713366
IGUAL percentile_90_word_length p=0.591007
5. scientific > non_scientific t = 2.630640  p = 0.008581  mean_word_length
6. scientific < non_scientific t = -14.626015  p = 4.428e-47  syllable_count
7. scientific < non_scientific t = -2.003444  p = 0.04524  avg_word_per_sentence
IGUAL LogicIfIncidence p=nan
8. scientific < non_scientific t = -2.010115  p = 0.04452  percentile_50_sentence_length
9. scientific < non_scientific t = -2.010115  p = 0.04452  median_sentence_length
IGUAL LogicAn

## unfavorable x mixed

In [14]:
tstudent(unfavorable_reviews, mixed_reviews)

1. scientific < non_scientific t = -6.231560  p = 8.977e-10  Unnamed: 0
IGUAL ConnectiveAdditiveIncidence p=0.496898
IGUAL sentence_sized_30 p=0.294586
IGUAL redability p=0.858921
IGUAL mean_sentence_length p=0.872871
IGUAL percentile_75_sentence_length p=0.797118
2. scientific < non_scientific t = -2.029472  p = 0.04286  percentile_75_word_length
3. scientific > non_scientific t = 2.114147  p = 0.03492  word_count
IGUAL ConnectiveLogicIncidence p=0.593622
IGUAL percentile_90_word_length p=0.068819
IGUAL mean_word_length p=0.255000
4. scientific > non_scientific t = 2.059502  p = 0.03988  syllable_count
IGUAL avg_word_per_sentence p=0.685544
IGUAL LogicIfIncidence p=nan
IGUAL percentile_50_sentence_length p=0.769062
IGUAL median_sentence_length p=0.769062
IGUAL LogicAndIncidence p=0.618282
IGUAL percentile_90_sentence_length p=0.331119
IGUAL median_word_length p=0.769062
IGUAL pronIncidence p=0.318726
5. scientific > non_scientific t = 2.894074  p = 0.003951  LogicOperatorsIncidence
IG